In [ ]:
import pandas as pd
data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
data = data.astype('string')

In [ ]:
def get_director(entity):
    data1 = data[data['director'].notna()]
    mask = data1['title'] == entity
    if(mask.any()):
        ans = 'The director of ' + entity + \
            ' is : ' + list(data1[mask]['director'])[0]
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans



In [ ]:
def get_directed(entity):
    data1 = data[data['title'].notna()]
    mask = data1['director'] == entity
    if(mask.any()):
        ans = entity + ' directed : ['
        for movie in list(data1[mask]['title']):
            ans += movie
            ans += ', '
        ans += ']'
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans


In [ ]:
def get_actors(entity):
    data1 = data[data['cast'].notna()]
    mask = data1['title'] == entity
    if(mask.any()):
        ans = entity + ' cast were : ['
        for movie in list(data1[mask]['cast']):
            ans += movie
            ans += ', '
        ans += ']'
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans

In [ ]:
def get_acted(entity):
    data1 = data[data['title'].notna()]
    data1 = data1[data1['cast'].notna()]
    ans = entity + ' acted in : ['
    cnt = 0
    cnt2 = 0
    for i in data1['cast']:
        if(entity in i.split(", ")):
            ans += data1.iloc[cnt]['title']
            ans += ', '
            cnt2 += 1
        cnt += 1
    ans += ']'
    if cnt2 == 0:
        return 'Sorry, couldn\'t find anything'
    else:
        return ans


In [ ]:
def get_year(entity):
    data1 = data[data['release_year'].notna()]
    mask = data1['title'] == entity
    if(mask.any()):
        ans = entity + ' was release on : ' + \
            list(data1[mask]['release_year'])[0]
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans


In [ ]:
def get_story(entity):
    data1 = data[data['description'].notna()]
    mask = data1['title'] == entity
    if(mask.any()):
        ans = entity + ' story is : ' + list(data1[mask]['description'])[0]
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans


In [ ]:

def get_duration(entity):
    data1 = data[data['duration'].notna()]
    mask = data1['title'] == entity
    if(mask.any()):
        ans = entity + ' duration is : ' + list(data1[mask]['duration'])[0]
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans


In [ ]:
def get_details(entity):
    mask = data['title'] == entity
    if(mask.any()):
        ans = entity + ' is a ' + list(data[mask]['type'])[0]
        ans += '\nwas released on (' + get_year(entity).split(' ')[-1] + ')'
        ans += '\ndirected by ' + get_director(entity).split(' ')[-1]
        ans += '\n' + get_story(entity)
        ans += '\nThe cast consist of ' + ' ' + get_actors(entity)
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans

In [ ]:
def get_type(entity):
    data1 = data[data['type'].notna()]
    mask = data1['title'] == entity
    if(mask.any()):
        ans = entity + ' is a ' + list(data1[mask]['type'])[0]
    else:
        ans = 'Sorry, couldn\'t find anything'
    return ans

In [ ]:
def main_function(intent, entity):
    if(intent == 'get_director'):
        return get_director(entity)
    elif(intent == 'get_directed'):
        return get_directed(entity)
    elif(intent == 'get_actors'):
        return get_actors(entity)
    elif(intent == 'get_acted'):
        return get_acted(entity)
    elif(intent == 'get_year'):
        return get_year(entity)
    elif(intent == 'get_story'):
        return get_story(entity)
    elif(intent == 'get_details'):
        return get_details(entity)
    elif(intent == 'get_duration'):
        return get_duration(entity)
    elif(intent == 'get_type'):
        return get_type(entity)
    elif(intent == 'get_details'):
        return get_details(entity)

In [ ]:
!pip install wit flask-ngrok pymessenger wit

In [ ]:
from logging import DEBUG
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from pymessenger import Bot
from wit import Wit

In [ ]:
PAGE_ACCESS_TOKEN = 'XXXXXXXXXXXXXXXX'
Wit_ACCESS_TOKEN = 'YYYYYYYYYYYYY'
VERIFICATION_TOKEN = 'ZZZZZZZZZZZ'


In [ ]:
bot = Bot(PAGE_ACCESS_TOKEN)


app = Flask(__name__)
run_with_ngrok(app)


In [ ]:
@app.route('/webhook', methods=['GET'])
def validate():
    if request.args.get('hub.mode', '') == 'subscribe' and request.args.get('hub.verify_token', '') == VERIFICATION_TOKEN:
        return request.args.get('hub.challenge', '')
    else:
        return 'Failed validation.'


@app.route('/webhook', methods=['POST'])
def webhook():
    data = request.get_json()
    process_request(data)
    return "ok", 200


In [ ]:
def process_request(data):
    # Check if value correponding to object key is "page"
    if data["object"] == "page":
        # loop on the list correponding to entry key
        for entry in data["entry"]:
            # loop on the list correponding to the messaging key
            for messaging_event in entry["messaging"]:
                # access the messege event:
                # (1) get sender and recipient IDs
                sender_id = messaging_event["sender"]["id"]
                recipient_id = messaging_event["recipient"]["id"]
                # (2) check messege type is simple messege type
                if messaging_event.get("message"):
                    # there is text key
                    if "text" in messaging_event["message"]:
                        messaging_text = messaging_event["message"]["text"]
                    else:
                        messaging_text = "no text"
                    get_wit(sender_id, messaging_text)



In [ ]:
def get_wit(sender_id, msg):
    client = Wit(Wit_ACCESS_TOKEN)
    # print(msg)
    resp = client.message("hi")  # msg)

    if len(resp.get('intents')) > 0:
        intent = resp.get('intents')[0].get('name')
        entity = list(resp.get('entities').values())[0][0].get('body')
        message = mmodule.main_function(intent, entity)
    else:
        message = "I'm not sure what to do"

    reply(sender_id, message)



In [ ]:
def reply(sender_id, response):
    bot.send_text_message(sender_id, response)


In [ ]:
if __name__ == "__main__":
    #app.run()
    pass